中国建网

In [ ]:
import time 
import pandas as pd
import networkx as nx
df = pd.read_csv('../中国航班数据/航班动态算法大赛原始数据/训练集/训练集/2015年5月到2017年5月航班动态数据.csv', encoding='GBK')

airport_data = pd.read_excel('../中国航班数据/航班动态算法大赛原始数据/训练集/训练集/机场城市对应表.xlsx')
airport_codes = set(airport_data['ID'])
airport_df = df[(df['出发机场'].isin(airport_codes)) & (df['到达机场'].isin(airport_codes))]

without_cancelled = airport_df[airport_df['航班是否取消'] == '正常'].dropna(how= 'all')
print(without_cancelled)

        出发机场 到达机场    航班编号      计划起飞时间      计划到达时间        实际起飞时间        实际到达时间  \
0        HGH  DLC  CZ6328  1453809600  1453817100  1.453813e+09  1.453819e+09   
1        SHA  XMN  FM9261  1452760800  1452767100  1.452763e+09  1.452768e+09   
2        CAN  WNZ  ZH9597  1453800900  1453807500  1.453802e+09  1.453807e+09   
3        SHA  ZUH  9C8819  1452120600  1452131100  1.452121e+09  1.452130e+09   
4        SHE  TAO   TZ185  1452399000  1452406800  1.452400e+09  1.452404e+09   
...      ...  ...     ...         ...         ...           ...           ...   
7518632  CTU  JUH  HO1270  1430799000  1430807400  1.430800e+09  1.430807e+09   
7518633  CKG  TNA  3U8507  1431301200  1431308700  1.431302e+09  1.431307e+09   
7518634  DLC  WUH  CZ6514  1433057100  1433066400  1.433067e+09  1.433074e+09   
7518636  PEK  SHA  CA1557  1431747000  1431754800  1.431748e+09  1.431754e+09   
7518637  CTU  LHW  CA4207  1432103400  1432108500  1.432105e+09  1.432108e+09   

           飞机编号 航班是否取消  
0 

In [3]:
without_cancelled['year'] = pd.to_datetime(without_cancelled['计划起飞时间'], unit='s').dt.year
without_cancelled = without_cancelled[without_cancelled['year'] == 2016]
without_cancelled.drop(columns= ['year'])

,出发机场,到达机场,航班编号,计划起飞时间,计划到达时间,实际起飞时间,实际到达时间,飞机编号,航班是否取消
0,HGH,DLC,CZ6328,1453809600,1453817100,1.453813e+09,1.453819e+09,1.0,正常
1,SHA,XMN,FM9261,1452760800,1452767100,1.452763e+09,1.452768e+09,2.0,正常
2,CAN,WNZ,ZH9597,1453800900,1453807500,1.453802e+09,1.453807e+09,3.0,正常
3,SHA,ZUH,9C8819,1452120600,1452131100,1.452121e+09,1.452130e+09,4.0,正常
4,SHE,TAO,TZ185,1452399000,1452406800,1.452400e+09,1.452404e+09,5.0,正常
...,...,...,...,...,...,...,...,...,...
3955270,LJG,KWE,8L9837,1483226400,1483230900,1.483226e+09,1.483230e+09,873.0,正常
3955843,CGO,HAK,Y87531,1483227000,1483237200,1.483227e+09,1.483237e+09,2961.0,正常
3957232,TAO,WNZ,SC4731,1483227000,1483234200,1.483227e+09,1.483233e+09,44.0,正常
3958380,JJN,WUH,MF8533,1483225500,1483232100,1.483227e+09,1.483232e+09,1028.0,正常


数据检查

In [4]:
without_cancelled.isna().any()
# tmpdf = without_cancelled[without_cancelled['实际起飞时间'].isna()]
# tmpdf

出发机场      False
到达机场      False
航班编号      False
计划起飞时间    False
计划到达时间    False
实际起飞时间     True
实际到达时间    False
飞机编号       True
航班是否取消    False
year      False
dtype: bool

年月日，时分秒，星期几，一年中第几天，夏令时标志

In [5]:
print(time.localtime(1471587000))

test = without_cancelled[without_cancelled['出发机场'] == 'ACX']
test2 = test[test['到达机场'] == 'PEK']
test2

time.struct_time(tm_year=2016, tm_mon=8, tm_mday=19, tm_hour=2, tm_min=10, tm_sec=0, tm_wday=4, tm_yday=232, tm_isdst=1)


,出发机场,到达机场,航班编号,计划起飞时间,计划到达时间,实际起飞时间,实际到达时间,飞机编号,航班是否取消,year
2365352,ACX,PEK,G52893,1471587000,1471605300,1.471587e+09,1.471605e+09,NaN,正常,2016


时间转化

In [ ]:
# def change_time(x):
#     if pd.isna(x):
#         return
#     return time.localtime(x)
# without_cancelled['计划起飞时间'] = without_cancelled['计划起飞时间'].apply(time.localtime)
# without_cancelled['计划到达时间'] = without_cancelled['计划到达时间'].apply(time.localtime)
# without_cancelled['实际起飞时间'] = without_cancelled['实际起飞时间'].apply(change_time)
# without_cancelled['实际到达时间'] = without_cancelled['实际到达时间'].apply(time.localtime)
# print(without_cancelled)

边

In [6]:
edges = list(set(zip(without_cancelled['出发机场'], without_cancelled['到达机场'])))
print(edges[:10])

[('YNT', 'HGH'), ('LYA', 'BAV'), ('ZHA', 'CAN'), ('WUH', 'LZO'), ('HET', 'CSX'), ('XIY', 'HAK'), ('CSX', 'BPE'), ('KMG', 'SJW'), ('BAV', 'TYN'), ('CAN', 'SWA')]


边-降落-起飞延迟dic

In [9]:
# edge_dep_arr_dic = {}
# for index, row in without_cancelled.iterrows():
#     if (row['出发机场'], row['到达机场']) in edge_dep_arr_dic:
#         if not pd.isna(row['实际起飞时间']):
#             if (row['实际起飞时间'] - row['计划起飞时间']) // 60 != 0:   
#                 edge_dep_arr_dic[(row['出发机场'], row['到达机场'])][0] += (row['实际起飞时间'] - row['计划起飞时间']) // 60
#                 edge_dep_arr_dic[(row['出发机场'], row['到达机场'])][2] += 1

#         if (row['实际到达时间'] - row['计划到达时间']) // 60 != 0:
#             edge_dep_arr_dic[(row['出发机场'], row['到达机场'])][1] += (row['实际到达时间'] - row['计划到达时间']) // 60
#             edge_dep_arr_dic[(row['出发机场'], row['到达机场'])][3] += 1
        
#     else:
#         edge_dep_arr_dic[(row['出发机场'], row['到达机场'])] = [0, 0, 0, 0]
#         if not pd.isna(row['实际起飞时间']):   
#             if (row['实际起飞时间'] - row['计划起飞时间']) // 60 != 0:
#                 edge_dep_arr_dic[(row['出发机场'], row['到达机场'])][0] += (row['实际起飞时间'] - row['计划起飞时间']) // 60
#                 edge_dep_arr_dic[(row['出发机场'], row['到达机场'])][2] += 1

#         if (row['实际到达时间'] - row['计划到达时间']) // 60 != 0:
#             edge_dep_arr_dic[(row['出发机场'], row['到达机场'])][1] += (row['实际到达时间'] - row['计划到达时间']) // 60
#             edge_dep_arr_dic[(row['出发机场'], row['到达机场'])][3] += 1

# print(edge_dep_arr_dic[('HGH', 'DLC')])

edge_dep_arr_dic = {}
# counter = 0
for index, row in without_cancelled.iterrows():
    if (pd.isna(row['实际起飞时间'])):
        continue
    if (row['出发机场'], row['到达机场']) in edge_dep_arr_dic:
        edge_dep_arr_dic[(row['出发机场'], row['到达机场'])][2] += 1
        edge_dep_arr_dic[(row['出发机场'], row['到达机场'])][0] += max(0, row['实际起飞时间'] - row['计划起飞时间']) //60
            

        # if not pd.isna(row['ARR_DELAY_NEW']):
        #     edge_dep_arr_dic[(row['出发机场'], row['到达机场'])][1] += row['ARR_DELAY_NEW']
        #     edge_dep_arr_dic[(row['出发机场'], row['到达机场'])][3] += 1
        # elif not pd.isna(row['CRS_ARR_TIME']) and not pd.isna(row['ARR_TIME']):
        #     row['ARR_DELAY_NEW'] = time_difference(row['CRS_ARR_TIME'], row['ARR_TIME'])
        #     counter  += 1
        #     edge_dep_arr_dic[(row['出发机场'], row['到达机场'])][1] += row['ARR_DELAY_NEW']
        #     edge_dep_arr_dic[(row['出发机场'], row['到达机场'])][3] += 1

    else:
        edge_dep_arr_dic[(row['出发机场'], row['到达机场'])] = [0, 0, 0, 0]
        edge_dep_arr_dic[(row['出发机场'], row['到达机场'])][2] += 1
        edge_dep_arr_dic[(row['出发机场'], row['到达机场'])][0] += max(0, row['实际起飞时间'] - row['计划起飞时间']) //60
    

        # if not pd.isna(row['ARR_DELAY_NEW']):
        #     edge_dep_arr_dic[(row['出发机场'], row['到达机场'])][1] += row['ARR_DELAY_NEW']
        #     edge_dep_arr_dic[(row['出发机场'], row['到达机场'])][3] += 1
        # elif not pd.isna(row['CRS_ARR_TIME']) and not pd.isna(row['ARR_TIME']):
        #     row['ARR_DELAY_NEW'] = time_difference(row['CRS_ARR_TIME'], row['ARR_TIME'])
        #     counter +=1
        #     edge_dep_arr_dic[(row['出发机场'], row['到达机场'])][1] += row['ARR_DELAY_NEW']
        #     edge_dep_arr_dic[(row['出发机场'], row['到达机场'])][3] += 1

print(edge_dep_arr_dic)
# print(counter)

{('HGH', 'DLC'): [107758.0, 0, 2760, 0], ('SHA', 'XMN'): [421998.0, 0, 8033, 0], ('CAN', 'WNZ'): [137609.0, 0, 4173, 0], ('SHA', 'ZUH'): [85512.0, 0, 1626, 0], ('SHE', 'TAO'): [75933.0, 0, 2782, 0], ('NKG', 'HAK'): [66229.0, 0, 1660, 0], ('LZO', 'PEK'): [28672.0, 0, 703, 0], ('WEH', 'HGH'): [12139.0, 0, 354, 0], ('JHG', 'LJG'): [28181.0, 0, 1683, 0], ('XIY', 'PEK'): [239727.0, 0, 8744, 0], ('KMG', 'ZUH'): [21593.0, 0, 666, 0], ('TAO', 'CGQ'): [64491.0, 0, 2350, 0], ('XNN', 'XIY'): [63401.0, 0, 4548, 0], ('KRY', 'URC'): [13175.0, 0, 746, 0], ('CAN', 'TNA'): [118310.0, 0, 2825, 0], ('HLD', 'HRB'): [9190.0, 0, 633, 0], ('LXA', 'YUS'): [11316.0, 0, 202, 0], ('WUH', 'HRB'): [39073.0, 0, 1015, 0], ('CSX', 'HAK'): [101700.0, 0, 3707, 0], ('PVG', 'LHW'): [93374.0, 0, 2110, 0], ('KMG', 'PEK'): [331363.0, 0, 7160, 0], ('WUH', 'KMG'): [121476.0, 0, 4595, 0], ('INC', 'HGH'): [33683.0, 0, 899, 0], ('HGH', 'CAN'): [494427.0, 0, 8387, 0], ('CKG', 'HAK'): [66471.0, 0, 2912, 0], ('CKG', 'PEK'): [224793

转无向，值全部简单相加

In [10]:
def undirected_f(l_dict):
    merged_dict = {}
    for key, value in l_dict.items():
        reverse_key = (key[1], key[0])
        if reverse_key in merged_dict:
            merged_dict[reverse_key] = [sum(x) for x in zip(merged_dict[reverse_key], value)]
        else:
            merged_dict[key] = value
    return merged_dict

edge_dep_arr_dic = undirected_f(edge_dep_arr_dic)
edge_dep_arr_dic

{('HGH', 'DLC'): [182563.0, 0, 5491, 0],
 ('SHA', 'XMN'): [886652.0, 0, 16463, 0],
 ('CAN', 'WNZ'): [244470.0, 0, 8345, 0],
 ('SHA', 'ZUH'): [207601.0, 0, 3305, 0],
 ('SHE', 'TAO'): [154663.0, 0, 5571, 0],
 ('NKG', 'HAK'): [124299.0, 0, 3325, 0],
 ('LZO', 'PEK'): [53490.0, 0, 1405, 0],
 ('WEH', 'HGH'): [17798.0, 0, 707, 0],
 ('JHG', 'LJG'): [84774.0, 0, 4813, 0],
 ('XIY', 'PEK'): [529006.0, 0, 17206, 0],
 ('KMG', 'ZUH'): [44179.0, 0, 1326, 0],
 ('TAO', 'CGQ'): [117580.0, 0, 4702, 0],
 ('XNN', 'XIY'): [147655.0, 0, 9436, 0],
 ('KRY', 'URC'): [27268.0, 0, 1468, 0],
 ('CAN', 'TNA'): [216165.0, 0, 5664, 0],
 ('HLD', 'HRB'): [23636.0, 0, 1268, 0],
 ('LXA', 'YUS'): [18110.0, 0, 405, 0],
 ('WUH', 'HRB'): [68489.0, 0, 2030, 0],
 ('CSX', 'HAK'): [187662.0, 0, 7409, 0],
 ('PVG', 'LHW'): [167285.0, 0, 4184, 0],
 ('KMG', 'PEK'): [604207.0, 0, 14373, 0],
 ('WUH', 'KMG'): [246110.0, 0, 9177, 0],
 ('INC', 'HGH'): [71973.0, 0, 1799, 0],
 ('HGH', 'CAN'): [933316.0, 0, 16779, 0],
 ('CKG', 'HAK'): [13786

边-延迟dic

In [11]:
edge_delay_dic ={}
for key, value in edge_dep_arr_dic.items():
    edge_delay_dic[key] = value[0] / value[2] #起飞延误

    # try:
    #     edge_delay_dic[key] = value[1] / value[3] #到达延误
    # except:
    #     print(value[1], value[3])
    #     edge_delay_dic[key] = 0

    # edge_delay_dic[key] = (value[0] + value[1]) / (value[2] + value[3])

print(edge_delay_dic)

{('HGH', 'DLC'): 33.247678018575854, ('SHA', 'XMN'): 53.85725566421673, ('CAN', 'WNZ'): 29.29538645895746, ('SHA', 'ZUH'): 62.814220877458396, ('SHE', 'TAO'): 27.762161191886555, ('NKG', 'HAK'): 37.38315789473684, ('LZO', 'PEK'): 38.071174377224196, ('WEH', 'HGH'): 25.173974540311175, ('JHG', 'LJG'): 17.613546644504467, ('XIY', 'PEK'): 30.745437638033245, ('KMG', 'ZUH'): 33.31749622926093, ('TAO', 'CGQ'): 25.006380263717567, ('XNN', 'XIY'): 15.648050021195422, ('KRY', 'URC'): 18.57493188010899, ('CAN', 'TNA'): 38.16472457627118, ('HLD', 'HRB'): 18.6403785488959, ('LXA', 'YUS'): 44.71604938271605, ('WUH', 'HRB'): 33.738423645320196, ('CSX', 'HAK'): 25.328924281279527, ('PVG', 'LHW'): 39.982074569789674, ('KMG', 'PEK'): 42.037640019480975, ('WUH', 'KMG'): 26.818132287239838, ('INC', 'HGH'): 40.007226236798225, ('HGH', 'CAN'): 55.624053876869894, ('CKG', 'HAK'): 23.671016483516482, ('CKG', 'PEK'): 31.366075824716887, ('CTU', 'LJG'): 22.640943720815926, ('PEK', 'CTU'): 39.09997466858059, (

延误传播网络

In [7]:
# sorted_without_cancelled = without_cancelled.sort_values(by=['飞机编号', '计划起飞时间'])
# edge_delay_dic ={}
# for index, row in sorted_without_cancelled.iterrows():
#     if index < len(sorted_without_cancelled) - 1:
#         next_row = sorted_without_cancelled.iloc[index + 1]
#         if pd.isna(row['实际起飞时间']) or pd.isna(next_row['实际起飞时间']): 
#             continue
#         if row['飞机编号'] == next_row['飞机编号']:
#             tat = (next_row['计划起飞时间'] - row['计划到达时间'])//60 - (next_row['实际起飞时间'] - row['实际到达时间'])//60
#             if tat>0 and next_row['实际到达时间']//60 - next_row['计划到达时间']//60>0:
#                 if (next_row['出发机场'], next_row['到达机场']) in edge_delay_dic:
#                     edge_delay_dic[(next_row['出发机场'], next_row['到达机场'])] += 1
#                 else:
#                     edge_delay_dic[(next_row['出发机场'], next_row['到达机场'])] = 1

edge_delay_dic ={}
without_cancelled.dropna(subset=['实际起飞时间'], inplace=True)
planeID_df = without_cancelled[without_cancelled['飞机编号'].notna()]
# no_planeID_df = without_cancelled[without_cancelled['飞机编号'].isna()]

y_sorted_df = planeID_df.sort_values(by=['飞机编号', '计划起飞时间']).reset_index(drop=True)
# n_sorted_df = no_planeID_df.sort_values(by=['航班编号', '计划起飞时间']).reset_index(drop=True)
for index, row in y_sorted_df.iterrows():
    if index < len(y_sorted_df) - 1:
        next_row = y_sorted_df.iloc[index + 1]
        if row['飞机编号'] == next_row['飞机编号'] and row['到达机场'] == next_row['出发机场']:
            stat = next_row['计划起飞时间']- row['计划到达时间']
            atat = next_row['实际起飞时间']- row['实际到达时间']
            tat = stat - atat
            if tat>0 and next_row['实际到达时间']> next_row['计划到达时间']:
            # tat = row['ARR_DELAY_NEW'] - next_row['DEP_DELAY_NEW']
            # if tat>0 and next_row['ARR_DELAY_NEW']>0:
                if (next_row['出发机场'], next_row['到达机场']) in edge_delay_dic:
                    edge_delay_dic[(next_row['出发机场'], next_row['到达机场'])] += 1
                else:
                    edge_delay_dic[(next_row['出发机场'], next_row['到达机场'])] = 1

print(edge_delay_dic[('HGH', 'DLC')])

146


csv转换格式

In [8]:
def rf(dic):
    tuple_list = []
    for key, value in dic.items():
        tuple_list.append((key[0], key[1], value))
    return tuple_list

csv_list = rf(edge_delay_dic)

平均延误网络保存

In [ ]:
df_edges = pd.DataFrame(csv_list, columns=['source', 'target', 'weight'])
df_edges_sorted = df_edges.sort_values(by=['source', 'target', 'weight'])
df_edges_sorted.to_csv('../建网结果/new_2016_中国_平均.csv', index=False)
print('done')

done


延误传播网络保存

In [ ]:
df_edges = pd.DataFrame(csv_list, columns=['source', 'target', 'weight'])
df_edges_sorted = df_edges.sort_values(by=['source', 'target', 'weight'])
df_edges_sorted.to_csv('../建网结果/new_2016_中国_传播.csv', index=False)
print('done')

done
